<a href="https://colab.research.google.com/github/flopezr2/WarehaousingLabEafit/blob/jueves28102021/WarehousingLab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pulp

     |████████████████████████████████| 41.2 MB 74 kB/s 


In [ ]:
import pulp as pulp
import pandas as pd
from pulp import *
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:

datos=pd.read_excel('/content/drive/MyDrive/Prueba_datos.xlsx',header=None,index_col=0)


In [ ]:
nom_posicion=[]
for i in range(1,datos.shape[1]+1):
  nombre= ('Pos ' + str(i))
  nom_posicion.append(nombre) 
datos.set_axis(nom_posicion,axis=1,inplace=True)

In [ ]:
datos.head()

,Pos 1,Pos 2,Pos 3,Pos 4,Pos 5,Pos 6,Pos 7,Pos 8,Pos 9,Pos 10
0,,,,,,,,,,
A,0,0,0,1,1,1,0,0,0,0
B,1,0,0,0,1,0,1,0,1,0
C,0,0,0,0,1,1,0,0,0,0
D,1,1,0,0,0,1,1,0,0,1
E,1,1,0,1,1,0,0,0,1,1


In [ ]:
#Conjuntos
conjunto_ordenes=datos.index
conjunto_posiciones=datos.columns
#conjunto_batches=['AA','BB','CC','DD','EE']


In [ ]:
ordenes=datos.shape[0]
cardinalidad=datos.shape[1]


In [ ]:
def matriz_afj(a_fj):
    num=0
    for orden in conjunto_ordenes:
      a_j={}
      for posicion in conjunto_posiciones:
        a_j[posicion]=datos[posicion].values[num]
      
      a_fj[orden]=a_j
      num+=1
    return a_fj

In [ ]:
#Parametros
p=ordenes #Numero de ordenes

q=3 # cantidad maxima de ordenes que puede contener un batche

conjunto_batches=np.arange(1,int(p/q)+2)

f_j=cardinalidad #Cardinalidad del conjunto de posiciones 

a_fj={} 
a_fj=matriz_afj(a_fj)# 1 si la orden f contiene la posicion j,0 sino.

In [ ]:
modelo=pulp.LpProblem('WarehousingLab2021',LpMinimize)

In [ ]:
#Variables
x_ij=pulp.LpVariable.dicts('x_ij',[(i,j) for i in conjunto_batches
                                         for j in conjunto_posiciones],0,1,cat='Binary') #1 si el batche i contiene la pisicion j, 0 sino.
y_if=pulp.LpVariable.dicts('y_if',[(i,f) for i in conjunto_batches
                                         for f in conjunto_ordenes],0,1,cat='Binary') #1 si el batche i contiene la orden f, 0 sino.


$función$ $Objetivo$

In [ ]:
modelo += lpSum(x_ij[(i,j)] for i in conjunto_batches for j in conjunto_posiciones)


**Restricciones**

In [ ]:
#Máximo 3 órdenes por batche
for i in conjunto_batches:
  modelo += lpSum(y_if[(i,f)] for f in conjunto_ordenes) <= q

#Debemos cumplir con las órdenes del día
modelo += lpSum(y_if[(i,f)] for i in conjunto_batches for f in conjunto_ordenes)==p

#Una orden en un solo batche
for f in conjunto_ordenes:
  modelo += lpSum(y_if[(i,f)] for i in conjunto_batches)==1

#Relacion de variables
for i in conjunto_batches:
  for j in conjunto_posiciones:
    modelo += lpSum(a_fj[f][j]*y_if[(i,f)] for f in conjunto_ordenes) <= x_ij[(i,j)]*f_j


In [ ]:
modelo.solve()
LpStatus[modelo.status]

'Optimal'

In [ ]:
pulp.value(modelo.objective )

23.0

In [ ]:
#Indica en que batche i va la orden  f
tol=0.0001
for i in conjunto_batches:
  for f in conjunto_ordenes:
    if y_if[(i,f)].varValue > tol:
      print("el batche ",i, 'contine la orden ',f )

el batche  1 contine la orden  I
el batche  2 contine la orden  B
el batche  2 contine la orden  E
el batche  2 contine la orden  G
el batche  3 contine la orden  A
el batche  3 contine la orden  C
el batche  3 contine la orden  F
el batche  4 contine la orden  D
el batche  4 contine la orden  H
el batche  4 contine la orden  J


In [ ]:
#Nos dice el batche que posiciones tiene que visitar
tol=0.0001
for i in conjunto_batches:
  for j in conjunto_posiciones:
    if x_ij[(i,j)].varValue > tol:
      print("el batche ",i, 'contine la posicion',j )

el batche  1 contine la posicion Pos 4
el batche  1 contine la posicion Pos 8
el batche  2 contine la posicion Pos 1
el batche  2 contine la posicion Pos 2
el batche  2 contine la posicion Pos 4
el batche  2 contine la posicion Pos 5
el batche  2 contine la posicion Pos 7
el batche  2 contine la posicion Pos 9
el batche  2 contine la posicion Pos 10
el batche  3 contine la posicion Pos 3
el batche  3 contine la posicion Pos 4
el batche  3 contine la posicion Pos 5
el batche  3 contine la posicion Pos 6
el batche  3 contine la posicion Pos 8
el batche  3 contine la posicion Pos 9
el batche  4 contine la posicion Pos 1
el batche  4 contine la posicion Pos 2
el batche  4 contine la posicion Pos 4
el batche  4 contine la posicion Pos 6
el batche  4 contine la posicion Pos 7
el batche  4 contine la posicion Pos 8
el batche  4 contine la posicion Pos 9
el batche  4 contine la posicion Pos 10


In [ ]:
#Numero de veces que se visita una posición
vis_posiciones={}
tol=0.0001
for i in conjunto_batches:
  for j in conjunto_posiciones:
    if x_ij[(i,j)].varValue > tol:
      if j in conjunto_posiciones and j not in vis_posiciones.keys():
        vis_posiciones[j]= 1
      elif j in vis_posiciones.keys():
        vis_posiciones[j] += 1

for posicion in vis_posiciones.keys():
  print(f'La posicion {posicion} fue visitada {vis_posiciones[posicion]} veces')


La posicion Pos 4 fue visitada 4 veces
La posicion Pos 8 fue visitada 3 veces
La posicion Pos 1 fue visitada 2 veces
La posicion Pos 2 fue visitada 2 veces
La posicion Pos 5 fue visitada 2 veces
La posicion Pos 7 fue visitada 2 veces
La posicion Pos 9 fue visitada 3 veces
La posicion Pos 10 fue visitada 2 veces
La posicion Pos 3 fue visitada 1 veces
La posicion Pos 6 fue visitada 2 veces


23